In [1]:
import networkx as nx, math, sys, numpy as np, pylcs

In [2]:
sys.setrecursionlimit(1000000)

In [3]:
g1o = nx.read_graphml('/home/exouser/results/amg_res/msg_size_512/n_procs_32/n_iters_20/ndp_0.0_0.2_0.8/run_000/event_graph.graphml')
g2o = nx.read_graphml('/home/exouser/results/amg_res/msg_size_512/n_procs_32/n_iters_20/ndp_0.0_0.2_0.8/run_001/event_graph.graphml')

In [4]:
len(g1o.nodes)

397056

In [5]:
# Hyperparameters
rev = False
norm_sens_high = False
skip_thresh = False
lcs = False
num_proc = 32

In [6]:
#Reverse Toggle
g1 = g1o
g2 = g2o
if rev:
    g1 = nx.reverse(g1)
    g2 = nx.reverse(g2)

In [7]:
d1 = dict(g1.nodes(data=True))
d2 = dict(g2.nodes(data=True))

In [8]:
proc_dict_1 = {i: [] for i in range(num_proc)}
proc_dict_2 = {i: [] for i in range(num_proc)}

In [9]:
for i in d1:
    proc_dict_1[int(d1[i]['process_id'])].append((i, d1[i]))
for i in d2:
    proc_dict_2[int(d2[i]['process_id'])].append((i, d2[i]))   

In [10]:
# MPI Dict
mpi_dict = {"MPI_Init": 'I', 'MPI_Barrier': 'B', 'MPI_Waitall': 'W', 'MPI_Isend':'S', 'MPI_Finalize':'F', 'MPI_Send':'s', 'MPI_Irecv':'R', 'MPI_Recv':'r', 'MPI_Wait':'w'}

In [11]:
# MPI Sequence
mpi_seq_1 = {i: "" for i in range(num_proc)}
mpi_seq_2 = {i: "" for i in range(num_proc)}
for i in proc_dict_1:
    for j in proc_dict_1[i]:
        mpi_seq_1[i] += mpi_dict[j[1]['mpi_function']]
for i in proc_dict_2:
    for j in proc_dict_2[i]:
        mpi_seq_2[i] += mpi_dict[j[1]['mpi_function']]

In [13]:
config_perms = {i: [] for i in range(num_proc)}     #Maps Run 1 to Run 2
lcs_list = []
if lcs:
    for i in proc_dict_1:
        lcs_list.append(pylcs.lcs_of_list(mpi_seq_1[i], [mpi_seq_2[i] for i in mpi_seq_2]))
for i in proc_dict_1:
    for j in proc_dict_2:
        if len(proc_dict_1[i]) != len(proc_dict_2[j]):    #Number of nodes threshold
            continue
        if lcs:
            if lcs_list[i][j] != min(lcs_list[i]):
                continue
            config_perms[i].append(j)
            continue
        if mpi_seq_1[i] != mpi_seq_2[j]:
            continue
        config_perms[i].append(j)

In [14]:
config_perms

{0: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 1: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 2: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 3: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 4: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 5: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,

In [15]:
scores_1 = {i:-1 for i in d1}
scores_2 = {i:-1 for i in d2}

In [16]:
MAX_TOPOL = max([d1[proc_dict_1[i][-1][0]]['logical_time'] for i in proc_dict_1])

In [17]:
def get_tree_struc_score(G, N, topol_map, scores):
    if scores[N] != -1:
        return
    #Reverse Toggle
    lvl = 0
    if rev:
        lvl = MAX_TOPOL-int(topol_map[N]['logical_time'])+1
    else:
        lvl = int(topol_map[N]['logical_time'])+1
    score = lvl**4 + lvl**3
    if norm_sens_high:
        score = math.log(np.float128(score))
    for neighbor in G[N]:
        get_tree_struc_score(G, neighbor, topol_map, scores)
        score += np.float128(scores[neighbor])
    if not norm_sens_high:
        scores[N] = math.log(score)
    else:
        scores[N] = score

In [18]:
#Reverse Toggle
for i in proc_dict_1:
    if rev:
        get_tree_struc_score(g1, proc_dict_1[i][-1][0], d1, scores_1)
    else:
        get_tree_struc_score(g1, proc_dict_1[i][0][0], d1, scores_1)
for i in proc_dict_2:
    if rev:
        get_tree_struc_score(g2, proc_dict_2[i][-1][0], d2, scores_2)
    else:
        get_tree_struc_score(g2, proc_dict_2[i][0][0], d2, scores_2)

In [19]:
for i in proc_dict_1:
    for j in proc_dict_1[i]:
        j[1]['score'] = scores_1[j[0]]
for i in proc_dict_2:
    for j in proc_dict_2[i]:
        j[1]['score'] = scores_2[j[0]]

In [20]:
for i in config_perms:
    new_config_perms = []
    errors = []
    for j in config_perms[i]:
        error = 0
        for k in range(len(proc_dict_1[i])):
            error += (proc_dict_1[i][k][1]['score'] - proc_dict_2[j][k][1]['score'])**2
        errors.append(error**0.5)
    min_error = min(errors)
    print(errors)
    for l in range(len(errors)):
        if errors[l] == min_error:
            new_config_perms.append(config_perms[i][l])
    config_perms[i] = new_config_perms


[0.0, 0.0007512260717864781, 0.0020158119499559856, 0.003774386301587096, 0.005490086783070639, 0.007129234112152523, 0.008692717888088747, 0.01018586058436056, 0.011614287147994605, 0.01298319292152987, 0.014297240121885933, 0.015560588586416206, 0.01677695265511564, 0.01794965796404356, 0.019081692005297007, 0.020175747655153634, 0.021234260287544594, 0.022259439381096664, 0.023253295494238487, 0.024217663366503883, 0.025154221779920688, 0.026064510703685186, 0.02694994615254105, 0.027811833113310817, 0.028651376832273378, 0.029469692705993342, 0.030267814977556883, 0.03104670440698838, 0.03180725505752656, 0.03255030031716178, 0.033276618256463844, 0.03398693640850604]
[0.0007512260717864781, 0.0, 0.0018036931414494998, 0.0035382332626125255, 0.005236938835405079, 0.006866128453533957, 0.008423318063332573, 0.009912168434225632, 0.011337486702351513, 0.012704034909051438, 0.0140162282676629, 0.01527807669208568, 0.016493198638790094, 0.017664855696215672, 0.018795990988436966, 0.019

In [21]:
config_perms

{0: [0],
 1: [1],
 2: [2],
 3: [3],
 4: [4],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9],
 10: [10],
 11: [11],
 12: [12],
 13: [13],
 14: [14],
 15: [15],
 16: [16],
 17: [17],
 18: [18],
 19: [19],
 20: [20],
 21: [21],
 22: [22],
 23: [23],
 24: [24],
 25: [25],
 26: [26],
 27: [27],
 28: [28],
 29: [29],
 30: [30],
 31: [31]}

In [ ]:
'''proc_neighbors_1 = np.zeros((num_proc, num_proc))
proc_neighbors_2 = np.zeros((num_proc, num_proc))

for i in proc_dict_1:
    for l in proc_dict_1[i]:
        for n in g1[l[0]]:
            proc_neighbors_1[i][int(d1[n]['process_id'])] = 1

for i in proc_dict_2:
    for l in proc_dict_2[i]:
        for n in g2[l[0]]:
            proc_neighbors_2[i][int(d2[n]['process_id'])] = 1'''

In [ ]:
'''meta_g1 = nx.DiGraph(proc_neighbors_1)
meta_g2 = nx.DiGraph(proc_neighbors_2)'''